In [68]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

In [70]:
import os
import pandas as pd

# Relative path from notebook/ to processed/
data_path = os.path.join("..", "data", "processed", "Ocean_Pearl_Recommendation_1.csv")

print("Looking for:", data_path)

raw_data = pd.read_csv(data_path)


Looking for: ..\data\processed\Ocean_Pearl_Recommendation_1.csv


In [72]:
raw_data

,Recipe ID,Dish Name,Category,Type,Cuisine,Rating,Prep Time (mins),Nutrients,Taste Profile & Tags,Ingredients,...,Review,Pairing Suggestions,Spice Level,Serving Size,Allergens,Availability,Chef's Special,Calories,Tags,Taste
0,R1,Tomato Soup,Starter,Veg,Continental,4.5,20,120 kcal / 4g protein / 20g carbs / 3g fat,"Savory, tangy; Tags: Vegetarian, Healthy, Quick","Tomatoes, cream, garlic, onion, butter, salt, ...",...,"Rich flavor profile, always satisfying.","French Fries, Coke",Mild,Medium,Dairy,Seasonal,Yes,120,Kids Special,Umami
1,R2,Mushroom Soup,Starter,Veg,Continental,4.4,25,150 kcal / 6g protein / 18g carbs / 4g fat,"Earthy, savory; Tags: Vegetarian, Healthy","Mushrooms, cream, butter, garlic, herbs",...,"A customer favorite, full of flavor.","French Fries, Coke",Mild,Large,Dairy,Always,Yes,150,Healthy,Spicy
2,R3,Chicken Soup,Starter,Non-Veg,Continental,4.6,30,160 kcal / 15g protein / 12g carbs / 5g fat,"Light, savory; Tags: Non-Veg, Healthy","Chicken, stock, garlic, onion, spices",...,"A customer favorite, full of flavor.","Garlic Bread, Soup",Mild,Small,NaN,Limited,Yes,160,Quick Bite,Umami
3,R4,Chicken Biryani,Rice Meal,Non-Veg,Indian,4.8,50,520 kcal / 25g protein / 70g carbs / 18g fat,"Spicy, aromatic; Tags: Non-Veg, Traditional","Rice, chicken, saffron, spices, onions",...,"Light, fresh, and very enjoyable.","Raita, Salad",Medium,Medium,NaN,Limited,Yes,520,Quick Bite,Spicy
4,R5,Mutton Biryani,Rice Meal,Non-Veg,Indian,4.7,60,600 kcal / 28g protein / 72g carbs / 22g fat,"Rich, spicy; Tags: Non-Veg, Traditional","Rice, mutton, saffron, spices, onions",...,"Rich flavor profile, always satisfying.","Raita, Salad",Medium,Large,NaN,Seasonal,Yes,600,Kids Special,Savory
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,R165,Paneer Tandoori Wrap,Snacks,Veg,Indian,4.6,18,360 kcal / 14g protein / 44g carbs / 12g fat,"Smoky, spicy; Tags: Wrap, Indian","Paneer tikka, onion, paratha, chutney",...,"Comforting and tasty, great with sides.","Plain Rice, Green Salad",Medium,Small,Dairy,Limited,Yes,360,Healthy,Spicy
128,R166,Smoked Chicken Sub,Snacks,Non-Veg,American,4.7,15,430 kcal / 28g protein / 44g carbs / 16g fat,"Smoky, filling; Tags: Sub, Fast Food","Bread, chicken, lettuce, cheese, BBQ sauce",...,"Comforting and tasty, great with sides.","French Fries, Coke",Mild,Large,NaN,Seasonal,Yes,430,Kids Special,Savory
129,R167,Veggie Delight Sub,Snacks,Veg,Continental,4.4,14,310 kcal / 9g protein / 42g carbs / 9g fat,"Fresh, crunchy; Tags: Sub, Healthy","Bread, cucumber, tomato, lettuce, sauces",...,A staple dish that never disappoints.,"Garlic Bread, Soup",Mild,Large,Dairy,Seasonal,No,310,Traditional,Spicy
130,R168,Avocado Toast,Breakfast,Veg,Continental,4.7,10,280 kcal / 8g protein / 34g carbs / 11g fat,"Creamy, fresh; Tags: Breakfast, Healthy","Bread, avocado, lemon, chili flakes",...,"Light, fresh, and very enjoyable.","Garlic Bread, Soup",Mild,Large,Dairy,Limited,Yes,280,Quick Bite,Spicy


## Select imp Features
1. Recipe ID
2. Dish Name
3. Category
4. Type
5. Cuisine
6. Rating
7. Ingredients
8. Price (₹)
9. 'Review'
10. Spice Level
11. Taste Profile
12. Tags

In [75]:
raw_data.columns

Index(['Recipe ID', 'Dish Name', 'Category', 'Type', 'Cuisine', 'Rating',
       'Prep Time (mins)', 'Nutrients', 'Taste Profile & Tags', 'Ingredients',
       'Price (₹)', 'Order Count', 'Time of Day', 'Review',
       'Pairing Suggestions', 'Spice Level', 'Serving Size', 'Allergens',
       'Availability', 'Chef's Special', 'Calories', 'Tags', 'Taste'],
      dtype='object')

In [77]:
data = raw_data[['Recipe ID', 'Dish Name', 'Category', 'Type', 'Cuisine', 'Rating',
        'Ingredients','Price (₹)', 'Review', 'Spice Level','Taste', 'Tags']]

In [79]:
data.head()

,Recipe ID,Dish Name,Category,Type,Cuisine,Rating,Ingredients,Price (₹),Review,Spice Level,Taste,Tags
0,R1,Tomato Soup,Starter,Veg,Continental,4.5,"Tomatoes, cream, garlic, onion, butter, salt, ...",180,"Rich flavor profile, always satisfying.",Mild,Umami,Kids Special
1,R2,Mushroom Soup,Starter,Veg,Continental,4.4,"Mushrooms, cream, butter, garlic, herbs",200,"A customer favorite, full of flavor.",Mild,Spicy,Healthy
2,R3,Chicken Soup,Starter,Non-Veg,Continental,4.6,"Chicken, stock, garlic, onion, spices",220,"A customer favorite, full of flavor.",Mild,Umami,Quick Bite
3,R4,Chicken Biryani,Rice Meal,Non-Veg,Indian,4.8,"Rice, chicken, saffron, spices, onions",440,"Light, fresh, and very enjoyable.",Medium,Spicy,Quick Bite
4,R5,Mutton Biryani,Rice Meal,Non-Veg,Indian,4.7,"Rice, mutton, saffron, spices, onions",490,"Rich flavor profile, always satisfying.",Medium,Savory,Kids Special


# Text Cleaning

In [82]:
nltk.download("stopwords")
nltk.download("wordnet")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shaik\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\shaik\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [84]:
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

In [86]:
def clean_text(text):
    if pd.isnull(text):
        return ""
    text = text.lower()                          # lowercase
    text = re.sub(r'[^a-z\s]', '', text)         # remove punctuation & numbers
    words = text.split()
    words = [lemmatizer.lemmatize(w) for w in words if w not in stop_words]
    return " ".join(words)

In [88]:
# Apply cleaning
data["Ingredients"] = data["Ingredients"].apply(clean_text)
data["Review"] = data["Review"].apply(clean_text)
data["Tags"] = data["Tags"].apply(clean_text)


# One-Hot Encode Categorical Features

In [91]:
categorical_cols = ["Type", "Cuisine", "Category", "Spice Level","Taste"]

In [93]:
data['Taste'].unique()

array(['Umami', 'Spicy', 'Savory', 'Tangy', 'Sweet'], dtype=object)

In [95]:
for col in categorical_cols:
    dummies = pd.get_dummies(data[col], prefix=col, drop_first=False)
    dummies = dummies.astype(bool)  # convert to True/False
    data.drop(columns=[col], inplace=True)
    data = pd.concat([data, dummies], axis=1)

#  Normalize Numeric Features

In [98]:
scaler = MinMaxScaler()
numeric_cols = ["Rating", "Price (₹)"]

In [100]:
data[numeric_cols] = scaler.fit_transform(data[numeric_cols])

#  Text Vectorization (TF-IDF)

In [103]:
tfidf_ingredients = TfidfVectorizer(max_features=500)
tfidf_tags = TfidfVectorizer(max_features=300)
tfidf_reviews = TfidfVectorizer(max_features=500)


In [105]:
ingredients_tfidf = tfidf_ingredients.fit_transform(data["Ingredients"].fillna(""))
tags_tfidf = tfidf_tags.fit_transform(data["Tags"].fillna(""))
reviews_tfidf = tfidf_reviews.fit_transform(data["Review"].fillna(""))

In [107]:
ingredients_df = pd.DataFrame(ingredients_tfidf.toarray(), columns=tfidf_ingredients.get_feature_names_out())
tags_df = pd.DataFrame(tags_tfidf.toarray(), columns=tfidf_tags.get_feature_names_out())
reviews_df = pd.DataFrame(reviews_tfidf.toarray(), columns=tfidf_reviews.get_feature_names_out())


#  Merge Final Features

In [115]:
data = pd.concat([data.reset_index(drop=True), ingredients_df, tags_df, reviews_df], axis=1)

In [117]:
cleaned_data = data.copy()

# Compute Cosine Similarity

In [120]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import OneHotEncoder

# Step 1: Drop non-feature columns
feature_cols = cleaned_data.drop(columns=["Recipe ID", "Dish Name", "Ingredients", "Review", "Tags"])

# Step 2: Separate numeric and categorical columns
numeric_cols = feature_cols.select_dtypes(include=['int64', 'float64'])
categorical_cols = feature_cols.select_dtypes(include=['object'])

# Step 3: One-hot encode categorical features (like Taste, Cuisine, etc.)
if not categorical_cols.empty:
    encoder = OneHotEncoder()
    encoded_cats = encoder.fit_transform(categorical_cols).toarray()
else:
    encoded_cats = None

# Step 4: Combine numeric + encoded categorical
import numpy as np
if encoded_cats is not None:
    final_features = np.hstack([numeric_cols.values, encoded_cats])
else:
    final_features = numeric_cols.values

# Step 5: Compute cosine similarity
cosine_sim = cosine_similarity(final_features)

print("Cosine similarity shape:", cosine_sim.shape)

Cosine similarity shape: (132, 132)


# Recommendation Function

In [123]:
def recommend_dishes(dish_name, top_n=5):
    if dish_name not in cleaned_data["Dish Name"].values:
        return f"❌ '{dish_name}' not found in dataset."
    
    # Get index of the dish
    idx = cleaned_data.index[cleaned_data["Dish Name"] == dish_name].tolist()[0]
    
    # Get similarity scores
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Sort by similarity
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Pick top N (excluding itself)
    sim_scores = sim_scores[1:top_n+1]
    
    # Get recommended dish IDs
    dish_indices = [i[0] for i in sim_scores]
    return cleaned_data.iloc[dish_indices][["Recipe ID", "Dish Name"]].to_dict(orient="records")

#  Test the System

In [126]:
recs = recommend_dishes("Paneer Butter Masala", top_n=5)
print("Recommended Dishes:\n", recs)

Recommended Dishes:
 [{'Recipe ID': 'R50', 'Dish Name': 'Kadai Paneer'}, {'Recipe ID': 'R63', 'Dish Name': 'Prawn Masala'}, {'Recipe ID': 'R73', 'Dish Name': 'Chili Paneer'}, {'Recipe ID': 'R87', 'Dish Name': 'Paneer Momos'}, {'Recipe ID': 'R36', 'Dish Name': 'Chole Bhature'}]
